Option chains
=======

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=12)

<IB connected to 127.0.0.1:7497 clientId=12>

Suppose we want to find the options on the SPX, with the following conditions:

* Use the next three monthly expiries;
* Use strike prices within +- 20 dollar of the current SPX value;
* Use strike prices that are a multitude of 5 dollar.

To get the current market value, first create a contract for the underlyer (the S&P 500 index):

In [2]:
spx = Index('SPX', 'CBOE')
ib.qualifyContracts(spx)

[Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX')]

Then get the ticker. Requesting a ticker can take up to 11 seconds.

In [3]:
[ticker] = ib.reqTickers(spx)
ticker

Ticker(contract=Index(conId=416904, symbol='SPX', exchange='CBOE', currency='USD', localSymbol='SPX'), time=datetime.datetime(2018, 5, 25, 17, 50, 1, 424100, tzinfo=datetime.timezone.utc), bid=2717.16, bidSize=1, ask=2717.92, askSize=1, last=2717.59, lastSize=0, high=2727.36, low=2716.72, close=2727.76, ticks=[TickData(time=datetime.datetime(2018, 5, 25, 17, 50, 1, 424100, tzinfo=datetime.timezone.utc), tickType=1, price=2717.16, size=1), TickData(time=datetime.datetime(2018, 5, 25, 17, 50, 1, 424100, tzinfo=datetime.timezone.utc), tickType=2, price=2717.92, size=1), TickData(time=datetime.datetime(2018, 5, 25, 17, 50, 1, 424100, tzinfo=datetime.timezone.utc), tickType=4, price=2717.59, size=0), TickData(time=datetime.datetime(2018, 5, 25, 17, 50, 1, 424100, tzinfo=datetime.timezone.utc), tickType=6, price=2727.36, size=0), TickData(time=datetime.datetime(2018, 5, 25, 17, 50, 1, 424100, tzinfo=datetime.timezone.utc), tickType=7, price=2716.72, size=0), TickData(time=datetime.datetime(2

Take the current market value of the ticker:

In [4]:
spxValue = ticker.marketPrice()
spxValue

2717.59

The following request fetches a list of option chains:

In [5]:
chains = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)

util.df(chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,SMART,416904,SPX,100,"{20180920, 20180719, 20191219, 20190117, 20180...","{2560.0, 1025.0, 2050.0, 3075.0, 4100.0, 2565...."
1,CBOE,416904,SPXW,100,"{20180817, 20180606, 20180727, 20180531, 20180...","{2560.0, 1025.0, 2050.0, 3075.0, 2565.0, 2570...."
2,SMART,416904,SPXW,100,"{20180817, 20180606, 20180727, 20180531, 20180...","{2560.0, 1025.0, 2050.0, 3075.0, 2565.0, 2570...."
3,CBOE,416904,SPX,100,"{20180920, 20180719, 20191219, 20190117, 20180...","{2560.0, 1025.0, 2050.0, 3075.0, 4100.0, 2565...."


These are four option chains that differ in ``exchange`` and ``tradingClass``. The latter is 'SPX' for the monthly and  'SPXW' for the weekly options. Note that the weekly expiries are disjoint from the monthly ones, so when interested in the weekly options the monthly options can be added as well.

In this case we're only interested in the montly options trading on SMART:

In [6]:
chain = next(c for c in chains if c.tradingClass == 'SPX' and c.exchange == 'SMART')
chain

OptionChain(exchange='SMART', underlyingConId=416904, tradingClass='SPX', multiplier='100', expirations={'20180920', '20180719', '20191219', '20190117', '20180816', '20190620', '20180614', '20181220', '20190314', '20201217'}, strikes={2560.0, 1025.0, 2050.0, 3075.0, 4100.0, 2565.0, 2570.0, 2060.0, 1550.0, 2575.0, 3600.0, 2580.0, 2070.0, 2585.0, 1050.0, 2075.0, 3100.0, 2590.0, 2080.0, 2595.0, 2085.0, 550.0, 1575.0, 2600.0, 2090.0, 2605.0, 2095.0, 2610.0, 1075.0, 2100.0, 3125.0, 2615.0, 2105.0, 2620.0, 2110.0, 1600.0, 2625.0, 2115.0, 2630.0, 2120.0, 2635.0, 1100.0, 2125.0, 3150.0, 2640.0, 2130.0, 2645.0, 2135.0, 600.0, 1625.0, 2650.0, 2140.0, 2655.0, 2145.0, 100.0, 1125.0, 2150.0, 2660.0, 3175.0, 2665.0, 2155.0, 2670.0, 2160.0, 1650.0, 2675.0, 3700.0, 2165.0, 2680.0, 2170.0, 2685.0, 1150.0, 2175.0, 3200.0, 2690.0, 2180.0, 2695.0, 2185.0, 650.0, 1675.0, 2700.0, 2190.0, 2705.0, 2195.0, 2710.0, 1175.0, 2200.0, 2715.0, 2205.0, 2720.0, 2210.0, 1700.0, 2725.0, 2215.0, 2730.0, 2220.0, 2735.0, 1

What we have here is the full matrix of expirations x strikes. From this we can build all the option contracts that meet our conditions:

In [7]:
strikes = [strike for strike in chain.strikes
        if strike % 5 == 0
        and spxValue - 20 < strike < spxValue + 20]
expirations = sorted(exp for exp in chain.expirations)[:3]
rights = ['P', 'C']

contracts = [Option('SPX', expiration, strike, right, 'SMART')
        for right in rights
        for expiration in expirations
        for strike in strikes]

ib.qualifyContracts(*contracts)
len(contracts)

48

In [8]:
contracts[0]

Option(conId=237923180, symbol='SPX', lastTradeDateOrContractMonth='20180614', strike=2700.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   180615P02700000', tradingClass='SPX')

Now to get the market data for all options in one go:

In [9]:
tickers = ib.reqTickers(*contracts)

tickers[0]

Ticker(contract=Option(conId=237923180, symbol='SPX', lastTradeDateOrContractMonth='20180614', strike=2700.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   180615P02700000', tradingClass='SPX'), time=datetime.datetime(2018, 5, 25, 17, 50, 2, 806952, tzinfo=datetime.timezone.utc), bid=19.4, bidSize=560, ask=19.9, askSize=279, last=20.3, lastSize=100, volume=1511, high=20.5, low=16.45, close=15.9, ticks=[], tickByTicks=[], domBids=[], domAsks=[], domTicks=[], bidGreeks=OptionComputation(impliedVol=0.10492681114485905, delta=-0.4019469624829666, optPrice=19.399999618530273, pvDividend=4.400254674460744, gamma=0.005647155960812031, vega=2.5253757317246706, theta=-0.5699540583961761, undPrice=2717.57), askGreeks=OptionComputation(impliedVol=0.10694319442007819, delta=-0.40357296844496793, optPrice=19.899999618530273, pvDividend=4.400254674460744, gamma=0.005546414620071337, vega=2.527989295906785, theta=-0.5824109444415362, undPrice=2717.57), lastGreeks=O

The option greeks are available from the ``modelGreeks`` attribute, and if there is a bid, ask resp. last price available also from ``bidGreeks``, ``askGreeks`` and ``lastGreeks``. For streaming ticks the greek values will be kept up to date to the current market situation.

In [10]:
ib.disconnect()